# Preprocessing Stage
## استدعاء المكتبات
هنا هنستدعي كل المكتبات اللي بنحتاجها خلال المشروع

In [1]:
import warnings
warnings.filterwarnings("ignore")
from pyarabic import araby
import os
from nltk.tag import StanfordPOSTagger
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer
import re
from nltk import ISRIStemmer
import pandas as pd
import numpy as np
from wordcloud import WordCloud
import arabic_reshaper
from bidi.algorithm import get_display
import matplotlib.pyplot as plt
import time

## 1- Tokenization Function
دي الفانكشن اللي بتقسم الجملة لكلمات <br>
**اللي استدعيناها فوق araby لاحظ استخدام مكتبة**

In [2]:
def Tokening(sample):
    tokens = araby.tokenize(sample)
    return tokens

👇مثال: هنا مثلا بنديله جملة وهو بيقطعها ويسميها توكينز

## 2- Part Of Speech Handling
دي الفانكشن اللي بتتعرف على اي اسم علم او حرف جر او كلمة غريبه وتحذفهم من الجملة وتخلي بس الصفات والافعال والاجزاء اللي تساعدنا في التقييم <br>
**اللي استدعيناها فوق لاستدعاء الموديل المطلوب StanfordPOSTagger لاحظ استخدام مكتبة**

In [3]:
java_path = "C:/Program Files/Java/jre1.8.0_251/bin/java.exe"
os.environ['JAVAHOME'] = java_path

In [4]:
jar = "stanford-postagger-full-2018-10-16/stanford-postagger.jar"
model = "stanford-postagger-full-2018-10-16/models/arabic.tagger"
pos_tagger = StanfordPOSTagger(model, jar, encoding = 'utf8')

In [5]:
def PartOfSpeech(tokens):
    pos_words = pos_tagger.tag(tokens)
    filtered_tokens = []
    unwanted_tags = {"CC", "NNP", "PRP", 'CD', "IN", 'UH', 'DT'}
    for word in pos_words:
        if word[0]:
            if word[0].split('/')[1] not in unwanted_tags:
                filtered_tokens.append(word[0].split('/')[0])
        elif word[1].split('/')[1] not in unwanted_tags:
            filtered_tokens.append(word[1].split('/')[0])
    return filtered_tokens

## 3- Named Entity Recognization (NER) Handling
دي الفانكشن اللي بتمسح اي زيادات من الكلمة علشان تكون كلها كلمات موحده ومفيش فرق في التشكيل <br>
**اللي استدعيناها فوق لاستدعاء الموديل المطلوب transformers -> pipeline لاحظ استخدام مكتبة**

In [6]:
# pipe = pipeline("ner", model="hatmimoha/arabic-ner")

In [7]:
tokenizer = AutoTokenizer.from_pretrained("hatmimoha/arabic-ner")
model = AutoModelForTokenClassification.from_pretrained("hatmimoha/arabic-ner")
Ner = pipeline("ner", model=model, tokenizer=tokenizer)

In [8]:
def NerDetective(sample):
    persons = []
    ner_obj = Ner(sample)
    unwanted_tags = {'B-PRICE', 'I-PRICE', "B-DISEASE", "I-DISEASE", 'B-PERSON', 'I-PERSON'}
    for i in range(len(ner_obj)):
        if ner_obj[i]["entity"] not in unwanted_tags:
            persons.append(ner_obj[i]["word"])
    return persons

In [9]:
def CleanNer(sample):
    words = []
    ner_detectived = NerDetective(sample)
    for word in ner_detectived:
        try:
            if word.startswith("##"):
                words[-1] = words[-1] + word.replace("##", "")
            else:
                words.append(word)
        except:
            pass
    return(words)

In [10]:
def RemoveNer(tokens, sample):
    cleaned_ner = CleanNer(sample)
    filltered_tokens = [token for token in tokens if token not in cleaned_ner]
    return filltered_tokens

## 4- Normalization Function
دي الفانكشن اللي بتمسح اي زيادات من الكلمة علشان تكون كلها كلمات موحده ومفيش فرق في التشكيل <br>
**اللي استدعيناها فوق re لاحظ استخدام مكتبة**

In [11]:
def Normalize(tokens):
    normalized_tokens = []
    for token in tokens:
        token = re.sub("[إأآا]", "ا", token)
        token = re.sub("ى", "ي", token)
#         token = re.sub("ة", "ه", token)
        token = re.sub("[\W\da-zA-Z]", "", token)
        token = re.sub("_", " ", token)
        token = araby.strip_diacritics(token)
        token = araby.strip_tatweel(token)
        if token != "":
            normalized_tokens.append(token)
    return normalized_tokens

## 5- Removing Stop Words Function
### استدعاء الكلمات المستبعده
هنا بناخد الكلمات الموجوده في الداتاسيت بتاعت الكلمات المستبعده (ستوب ووردس)

In [12]:
def StopWords():
    sample = open('DataSets\stop_words.txt', 'r', encoding='utf-8')
    sample_Words = str(sample.read())
    stopwords = Tokening(sample_Words)
    return stopwords

### مسح الكلمات المستبعده من الجملة المعطاه
وهنا بنشوف لو الكلمة موجوده فيها نمسحها

In [13]:
def RemoveStopWords(tokens):
    stop_words = StopWords()
    filtered_tokens = [token for token in tokens if token not in stop_words]
    return filtered_tokens

## 6- Stemming Function
هنا بنرجع الكلمة لأصلها <br>
**اللي استدعيناها فوق ISRIStemmer لاحظ استخدام مكتبة**

In [14]:
def Stemming(tokens):
    stemmer = ISRIStemmer()
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return stemmed_tokens

## 7- Restore tokens to sentence function
هنا بنرجع الكلمات الى جمل<br>

In [15]:
def ToSentence(tokens):
    sentence = ' '.join(token for token in tokens)
    return sentence

## 8- Generating Word Cloud Figure
هنا بنعمل خريطة للكلمات الاكثر استخداماً في الجمل او المقال<br>

In [16]:
def GenWordCloud(df):
    text = " ".join(sentence for sentence in df['cleaned_text'])
    # Reshape and display the Arabic text
    reshaped_text = arabic_reshaper.reshape(text)
    bidi_text = get_display(reshaped_text)
    # Generate the word cloud
    wordcloud = WordCloud(font_path='tahomabd.ttf',
                          width=1600, height=800,
                          margin=0, background_color="white").generate(bidi_text)
    # Plot the word cloud
    plt.figure(figsize=(20, 10))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.savefig("fig.png")

### Cleaning & Preprocessing Functions

In [17]:
def PreProcess(df):
    print("PreProcessing Started!")
    start_time = time.perf_counter()
    df['cleaned_text'] = df['text'].apply(Tokening)
    end_time = time.perf_counter()
    print("Tokenization complete!", end_time - start_time)
    start_time = time.perf_counter()
    df['cleaned_text'] = df['cleaned_text'].apply(Normalize)
    end_time = time.perf_counter()
    print("Normalization complete!", end_time - start_time)
    start_time = time.perf_counter()
    # df['cleaned_text'] = df['cleaned_text'].apply(PartOfSpeech)
    end_time = time.perf_counter()
    print("POS filtering complete!", end_time - start_time)
    df['preprocessed_text'] = df['cleaned_text'].apply(ToSentence)
    start_time = time.perf_counter()
    # df['cleaned_text'] = df.apply(lambda x: RemoveNer(x['cleaned_text'], x['preprocessed_text']), axis=1)
    end_time = time.perf_counter()
    print("NER filtering complete!", end_time - start_time)
    start_time = time.perf_counter()
    df['preprocessed_text'] = df['cleaned_text'].apply(Stemming)
    end_time = time.perf_counter()
    print("Stemming complete!", end_time - start_time)
    start_time = time.perf_counter()
    df['preprocessed_text'] = df['preprocessed_text'].apply(RemoveStopWords)
    end_time = time.perf_counter()
    print("Stopword removal complete!", end_time - start_time)
    df['cleaned_text'] = df['cleaned_text'].apply(ToSentence)
    df['preprocessed_text'] = df['preprocessed_text'].apply(ToSentence)
    return df

# Reading Sentences stage

ARSAS

In [18]:
file_path = "DataSets\ArSAS.txt"
# headers = ['text']
df_ara = pd.read_csv(file_path, encoding="utf-8", delimiter='\t')
# df.columns = headers

In [19]:
df_ara = df_ara[df_ara['Sentiment_label'] != "Mixed"]
df_ara = df_ara[df_ara['Sentiment_label_confidence'] >= .5]

In [20]:
df_ara.drop(columns={'#Tweet_ID', 'Topic', 'Sentiment_label_confidence', 'Speech_act_label', 'Speech_act_label_confidence'},
        inplace=True)
df_ara.rename(columns={'Tweet_text': 'text', 'Sentiment_label' : 'class'},
         inplace=True)

In [21]:
df_ara['class'] = df_ara['class'].replace('Neutral', 'NEU').replace('Negative', 'NEG').replace('Positive', 'POS')

In [22]:
df_ara = PreProcess(df_ara)

PreProcessing Started!
Tokenization complete! 1.312864899984561
Normalization complete! 6.24091920000501
POS filtering complete! 3.00002284348011e-07
NER filtering complete! 9.00006853044033e-07
Stemming complete! 3.45221620000666
Stopword removal complete! 32.961478999990504


In [23]:
df_ara.dropna(inplace=True)
df_ara.drop_duplicates(inplace=True)
df_ara.reset_index(inplace=True, drop=True)

TWEETS

In [25]:
file_path = "DataSets\Tweets.txt"
headers = ['text', 'class']
df_tweets = pd.read_csv(file_path, encoding="utf-8", delimiter='\t')
df_tweets.columns = headers

In [26]:
df_tweets['class'].replace('NEUTRAL', 'NEU', inplace=True)

In [27]:
df_tweets = df_tweets[df_tweets['class'] != "OBJ"]

In [28]:
df_tweets = PreProcess(df_tweets)

PreProcessing Started!
Tokenization complete! 0.3343369000358507
Normalization complete! 0.8693106999853626
POS filtering complete! 7.00005330145359e-07
NER filtering complete! 1.00000761449337e-06
Stemming complete! 0.5291897000279278
Stopword removal complete! 6.650740400014911


In [29]:
df_tweets.dropna(inplace=True)
df_tweets.drop_duplicates(inplace=True)
df_tweets.reset_index(inplace=True, drop=True)

LOCAL

In [30]:
file_path = "DataSets\TrainingDataSet.csv"
# headers = ['text', 'class']
df_local = pd.read_csv(file_path, encoding="utf-8")
# df_tweets.columns = headers

In [31]:
df_local = PreProcess(df_local)

PreProcessing Started!
Tokenization complete! 0.17022289999295026
Normalization complete! 0.7585197000298649
POS filtering complete! 4.00003045797348e-07
NER filtering complete! 1.200009137392044e-06
Stemming complete! 0.8485243999748491
Stopword removal complete! 22.13287390000187


In [32]:
df_temp = df_local['class']
df_local['class'] = df_local['text']
df_local['text'] = df_temp
df_local.rename(columns={'class':'text','text':'class'},inplace=True)

In [33]:
df_local.dropna(inplace=True)
df_local.drop_duplicates(inplace=True)
df_local.reset_index(inplace=True, drop=True)

In [34]:
df_ara.sample(3)

,text,class,cleaned_text,preprocessed_text
14586,قطر مع الاخوان وتركيا الي هم من اشعل فتنة الرب...,NEG,قطر مع الاخوان وتركيا الي هم من اشعل فتنة الرب...,قطر اخو وتر شعل فتن عرب
16373,عقبال ما تحاسبوا الخاين اللي باع الأرض و فرط ف...,NEG,عقبال ما تحاسبوا الخاين اللي باع الارض و فرط ف...,عقبال خين باع ارض و فرط ياه شرب شطر هيف سد الن...
10037,#المقرحي: الضابط المفصول المُتهم في حادث #الوا...,NEU,المقرحي الضابط المفصول المتهم في حادث الواحات ...,قرح ضبط فصل تهم واح صاب بلب فصل


In [35]:
df_local.sample(3)

,text,class,cleaned_text,preprocessed_text
6790,كل سنة وأنت طيب,POS,كل سنة وانت طيب,سنة ونت طيب
7682,متدين,POS,متدين,متد
4406,مش مبسوطه,NEG,مش مبسوطه,مش بسط


In [36]:
df_tweets.sample(3)

,text,class,cleaned_text,preprocessed_text
393,1540 _ عقاب المخرب هو القتل عند الحقراء عديمي ...,NEG,عقاب المخرب هو القتل عند الحقراء عديمي الانس...,عقب خرب قتل حقراء سان كفي نطق دنء بتع حيو عق...
330,ذو العقل يشقى في النعيم بعقلة و أخو الجهالة في...,NEU,ذو العقل يشقي في النعيم بعقلة و اخو الجهالة في...,عقل يشق نعم عقل و اخو جهل شقو نعم اروع ابيات ا...
642,مشايخ البيادة عكس اسمائهم احمد الطيب اقسي قلب ...,NEG,مشايخ البيادة عكس اسمائهم احمد الطيب اقسي قلب ...,شاخ عكس سمئ حمد طيب اقس قلب سلم الغ صلة حمد كر...


In [44]:
df = pd.concat([df_ara,df_local,df_tweets])

In [45]:
df_ara.to_csv("DataSets\CleanedTrainingDataSetARA.csv", encoding="utf-8",index=False)

In [46]:
df_local.to_csv("DataSets\CleanedTrainingDataSetLOCAL.csv", encoding="utf-8",index=False)

In [47]:
df_tweets.to_csv("DataSets\CleanedTrainingDataSetTWEETS.csv", encoding="utf-8",index=False)

In [48]:
df.to_csv("DataSets\CleanedTrainingDataSet.csv", encoding="utf-8",index=False)